In [1]:
import pandas as pd


In [50]:
c1 = pd.read_csv("data/CAERS_ASCII_2004_2013_0.csv",encoding='cp1252')
c2 = pd.read_csv("data/CAERS_ASCII_11_14_to_12_17.csv",encoding="unicode_escape")
c3 = pd.read_csv("data/CAERS_ASCII_11_18_to_3_20.csv",encoding='cp1252')



In [51]:
print(len(c1),len(c2),len(c3))

50055 50440 30766


In [52]:
aggReports = pd.concat([c1, c2, c3])

In [53]:
aggReports.head()

,Report ID,CAERS Created Date,Date of Event,Product Type,Product,Product Code,Description,Patient Age,Age Units,Sex,MedDRA Preferred Terms,Outcomes,Medra Preferred Terms
0,65335,1/1/2004,11/24/2003,SUSPECT,ENFAMIL LIPIL BABY FORMULA,40,Baby Food Products,3.0,month(s),NaN,"VOMITING, GASTROINTESTINAL DISORDER",Other Outcome,NaN
1,65345,1/1/2004,12/21/2003,SUSPECT,"FRITO LAY FUNYUNS ONION FLAVOR, ONION RINGS",7,Snack Food Item,10.0,year(s),M,CHOKING,Other Outcome,NaN
2,65350,1/1/2004,NaN,SUSPECT,GRAPE,20,Fruit/Fruit Prod,NaN,NaN,NaN,"DEATH, CHOKING",Death,NaN
3,65336,1/1/2004,NaN,SUSPECT,ENFIMIL LIPIL BABY FORMULA,40,Baby Food Products,NaN,NaN,NaN,"PHYSICAL EXAMINATION, GASTROINTESTINAL DISORDER",Patient Visited Healthcare Provider,NaN
4,65333,1/1/2004,NaN,SUSPECT,KROGER CLASSIC CREAM-DE-MINT CANDY MINT CHIP I...,13,Ice Cream Prod,NaN,NaN,NaN,"NAUSEA, DYSGEUSIA, DIARRHOEA",Patient Visited ER,NaN


In [54]:
(aggReports.isnull().sum()/aggReports.shape[0])*100

Report ID                  0.000000
CAERS Created Date         0.000000
Date of Event             38.439445
Product Type               0.000000
Product                    0.002286
Product Code               0.495958
Description                0.495958
Patient Age               39.411554
Age Units                 39.314038
Sex                       40.522318
MedDRA Preferred Terms    61.866053
Outcomes                   0.000000
Medra Preferred Terms     38.951402
dtype: float64

In [57]:
len(aggReports)

131261

In [58]:
aggReports_no_dupes = aggReports.copy()
aggReports_no_dupes.drop_duplicates(subset = 'Report ID',inplace=True)

len(aggReports_no_dupes)

91776

In [60]:
(aggReports_no_dupes.isnull().sum()/aggReports.shape[0])*100

Report ID                  0.000000
CAERS Created Date         0.000000
Date of Event             28.853201
Product Type               0.000000
Product                    0.001524
Product Code               0.474627
Description                0.474627
Patient Age               33.466909
Age Units                 33.391487
Sex                       34.260748
MedDRA Preferred Terms    41.396150
Outcomes                   0.000000
Medra Preferred Terms     29.271452
dtype: float64

In [68]:
aggReports_no_dupes.nunique()

Report ID                 91776
CAERS Created Date         4927
Date of Event              6503
Product Type                  2
Product                   42497
Product Code                 48
Description                  50
Patient Age                 107
Age Units                     5
Sex                           4
MedDRA Preferred Terms    18984
Outcomes                    420
Medra Preferred Terms     23531
time_diff                  4785
time_group                    6
dtype: int64

In [63]:
aggReports_no_dupes["CAERS Created Date"] = pd.to_datetime(aggReports_no_dupes["CAERS Created Date"])
aggReports_no_dupes["Date of Event"] = pd.to_datetime(aggReports_no_dupes["Date of Event"])

aggReports_no_dupes["time_diff"] = (aggReports_no_dupes['CAERS Created Date']-aggReports_no_dupes['Date of Event']).dt.days

# Group the Time difference by different periods
time_list = (0,180,366,1098,1830,3660)
time_cat = ['less than 6 Months','less than 1 year','1 to 3 years','3 to 5 years','5 to 10 Years']
aggReports_no_dupes['time_group'] = pd.cut(aggReports_no_dupes['time_diff'],time_list,labels= time_cat)

aggReports_no_dupes['time_group'] = aggReports_no_dupes['time_group'].astype('str')
aggReports_no_dupes['time_group'] = aggReports_no_dupes['time_group'].replace('nan','Unknown')

In [72]:
aggReports_no_dupes.groupby('time_group')['Report ID'].count()

time_group
1 to 3 years           5304
3 to 5 years           2627
5 to 10 Years          2931
Unknown               40120
less than 1 year       3546
less than 6 Months    37248
Name: Report ID, dtype: int64